SETTING UP THE ENV

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
load_dotenv(find_dotenv())
GEMINI_API_KEY = os.getenv("GEMINI_KEY")


IMPORTING DOC IN LOADER

In [4]:


from langchain_community.document_loaders import PyPDFLoader

pdf_path =  "G:\Data Analyst CN\Python\LangChain\Byte_Pair_Encoding.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()
documents



<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Dell\AppData\Local\Temp\ipykernel_9560\3273384520.py:3: SyntaxWarning: invalid escape sequence '\D'
  pdf_path =  "G:\Data Analyst CN\Python\LangChain\Byte_Pair_Encoding.pdf"


[Document(metadata={'producer': 'GPL Ghostscript GIT PRERELEASE 9.08', 'creator': 'dvips(k) 5.991 Copyright 2011 Radical Eye Software', 'creationdate': '2016-06-13T01:31:02-04:00', 'moddate': '2016-06-13T01:31:02-04:00', 'title': 'arXiv:1508.07909v5  [cs.CL]  10 Jun 2016', 'source': 'G:\\Data Analyst CN\\Python\\LangChain\\Byte_Pair_Encoding.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='arXiv:1508.07909v5  [cs.CL]  10 Jun 2016\nNeural Machine Translation of Rare Words with Subword Units\nRico Sennrichand Barry Haddow and Alexandra Birch\nSchool of Informatics, University of Edinburgh\n{rico.sennrich,a.birch}@ed.ac.uk,bhaddow@inf.ed.ac.uk\nAbstract\nNeural machine translation (NMT) mod-\nels typically operate with a ﬁxed vocabu-\nlary, but translation is an open-vocabulary\nproblem. Previous work addresses the\ntranslation of out-of-vocabulary words by\nbacking off to a dictionary. In this pa-\nper, we introduce a simpler and more ef-\nfective approach, making th

EMBEDDING THE DOC

In [5]:

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

textsplitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = textsplitter.split_documents(documents)
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore

C:\Users\Dell\AppData\Local\Temp\ipykernel_9560\3684017633.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


SAVE THE VECTORSTORE

In [6]:
vectorstore.save_local("byte_pair_encoding_vectorstore")
# Load the vectorstore from the saved location
# vectorstore = FAISS.load_local("byte_pair_encoding_vectorstore", embeddings)

EMBEDDING THE QUESTION

In [7]:
my_question = "What is Byte Pair Encoding?"
embedded_question = embeddings.embed_query(my_question)
embedded_question

[-0.3442440927028656,
 1.855553388595581,
 -3.0146970748901367,
 -1.1159172058105469,
 1.149372935295105,
 0.27392542362213135,
 -1.534379005432129,
 -0.06926979124546051,
 -0.8770150542259216,
 -0.915999174118042,
 -0.7844842076301575,
 1.9602628946304321,
 -0.24846473336219788,
 0.18598106503486633,
 0.15666267275810242,
 -0.8490591645240784,
 -1.0020636320114136,
 -0.5475313067436218,
 -0.027988990768790245,
 -1.213639736175537,
 -0.6544400453567505,
 0.929191529750824,
 0.24361839890480042,
 -0.819237232208252,
 1.7831906080245972,
 0.5073801875114441,
 -0.29540932178497314,
 0.43027356266975403,
 -0.18504567444324493,
 0.2895299792289734,
 1.6135669946670532,
 -0.16319189965724945,
 0.4346666634082794,
 -0.46110472083091736,
 -1.2377030849456787,
 -0.6822565793991089,
 0.5361803770065308,
 -1.1412800550460815,
 -0.5721873044967651,
 0.4100281298160553,
 -0.020976098254323006,
 0.06717590987682343,
 -0.9983847141265869,
 -0.2013765424489975,
 1.3185852766036987,
 0.1296511590480804

MATCHING THE QUESTION WITH THE VECTOR STORE

In [10]:
Retrieve_docs = vectorstore.similarity_search_by_vector(embedded_question, k=3)
Retrieve_docs[0].page_content

'increase time and space efﬁciency, and to allow for\ntranslation without back-off models. At the same\ntime, we also want a compact representation of the\ntext itself, since an increase in text length reduces\nefﬁciency and increases the distances over which\nneural models need to pass information.\nA simple method to manipulate the trade-off be-\ntween vocabulary size and text size is to use short-\nlists of unsegmented words, using subword units\nonly for rare words. As an alternative, we pro-\npose a segmentation algorithm based on byte pair\nencoding (BPE), which lets us learn a vocabulary\nthat provides a good compression rate of the text.\n3.2 Byte Pair Encoding (BPE)\nByte Pair Encoding (BPE) (Gage, 1994) is a sim-\nple data compression technique that iteratively re-\nplaces the most frequent pair of bytes in a se-\nquence with a single, unused byte. We adapt this\nalgorithm for word segmentation. Instead of merg-\ning frequent pairs of bytes, we merge characters or\ncharacter 

Merging all the retrieved_docs in a single string

In [11]:
all_context = ""
for doc in Retrieve_docs:
    all_context += doc.page_content + "\n"

CREATING THE RAG PROMPT

In [12]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

c:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

Constructing the message with Context and Question

In [13]:
message = prompt.invoke(
    {
        "question": my_question,
        "context": all_context
    }
)
message

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is Byte Pair Encoding? \nContext: increase time and space efﬁciency, and to allow for\ntranslation without back-off models. At the same\ntime, we also want a compact representation of the\ntext itself, since an increase in text length reduces\nefﬁciency and increases the distances over which\nneural models need to pass information.\nA simple method to manipulate the trade-off be-\ntween vocabulary size and text size is to use short-\nlists of unsegmented words, using subword units\nonly for rare words. As an alternative, we pro-\npose a segmentation algorithm based on byte pair\nencoding (BPE), which lets us learn a vocabulary\nthat provides a good compression rate of the text.\n3.2 Byte P

Invoking the LLM(Ollama) with the message

In [14]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2:1b", temperature=0.1)
result = llm.invoke(message)

C:\Users\Dell\AppData\Local\Temp\ipykernel_9560\2646123809.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:1b", temperature=0.1)


In [15]:
print("Answer:", result)

Answer: Byte Pair Encoding (BPE) is a simple data compression technique that iteratively replaces the most frequent pair of bytes in a sequence with a single, unused byte. This process is repeated for each character or character sequence in the input text, allowing for efficient and compact representation of the text while maintaining a good trade-off between vocabulary size and text size. BPE can be used to learn a vocabulary that provides a good compression rate of the text by merging frequent pairs of bytes into single symbols.
